In [4]:
from PIL import Image
from pillow_heif import register_heif_opener
import torch
import torch.nn as nn
import numpy as np
from torchvision.utils import save_image, make_grid


register_heif_opener()

image = Image.open('data/IMG_1432.HEIC')


print(image.size)
print(image.mode)

(16282, 3834)
RGB


In [2]:
DEVICE = 'mps'


(16282, 3834)